<a href="https://colab.research.google.com/github/agbleze/production_ready-applied_DL/blob/main/W%26B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.8 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=40b6fa2eaf988f3cd0540e5bda915f474c384f213708efc336da7932f411bf0b
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [2]:
import wandb

In [3]:
import numpy as np

In [4]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [22]:
run= wandb.init(project="example-DL-Book", name="run-1")#, entity="tpalczew")

In [23]:
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 100,
  "batch_size": 128
}

In [24]:
!pwd

/content


In [25]:
import os
file_path = os.getcwd()

In [7]:
#file_path = '/Users/tpalczew/WB/example/'

In [26]:
wandb.log_artifact(file_path, name='new_artifact', type='my_dataset')

wandb: Adding directory to artifact (/content)... Done. 0.2s


<Artifact new_artifact>

In [27]:
artifact = wandb.Artifact('new_artifact', type='my_dataset')

In [28]:
artifact

<Artifact new_artifact>

In [29]:
!mkdir dataset

mkdir: cannot create directory ‘dataset’: File exists


In [30]:
#artifact.add_dir('/Users/tpalczew/dataset/')
artifact.add_dir(f'{file_path}/dataset')

wandb: Adding directory to artifact (/content/dataset)... Done. 0.0s


In [31]:
wandb.log_artifact(artifact)

<Artifact new_artifact>

In [33]:
#import wandb
#run = wandb.init()
artifact = run.use_artifact('datasiast/example-DL-Book/new_artifact:v3', type='my_dataset')

# datasiast/example-DL-Book/new_artifact:v3
artifact_dir = artifact.download()


#import wandb
#run = wandb.init()
#artifact = run.use_artifact('datasiast/example-DL-Book/new_artifact:v3', type='my_dataset')
#artifact_dir = artifact.download()

In [34]:
print(artifact_dir)

./artifacts/new_artifact:v3


In [35]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print(tf.__version__) # should return tf 2.x version

2.12.0


In [36]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [37]:
model = keras.Sequential()
model.add(keras.Input(shape=(50,)))
model.add(layers.Dense(128, activation="relu", name="layer1"))
model.add(layers.Dense(256, activation="relu", name="layer2"))
model.add(layers.Dense(1, activation="sigmoid", name="layer3"))

In [38]:
from wandb.keras import WandbCallback

In [39]:
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 100,
  "batch_size": 128
}

In [40]:
print(wandb.config)

{'learning_rate': 0.001, 'epochs': 100, 'batch_size': 128}


In [41]:
optimizer = tf.keras.optimizers.Nadam(learning_rate=wandb.config['learning_rate'], beta_1=0.9, beta_2=0.999, clipnorm=1.0)

model.compile(loss = "binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

In [42]:
fake_data = np.random.rand(1000,50)

In [43]:
fake_labels = np.random.randint(2, size=(1000,1))

In [44]:
fake_data_val = np.random.rand(1000,50)

In [45]:
fake_labels_val = np.random.randint(2, size=(1000,1))

In [46]:
logging_callback = WandbCallback(log_evaluation=True)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [47]:
model.fit(x=fake_data, y=fake_labels,
          epochs=wandb.config['epochs'], batch_size=wandb.config['batch_size'],
          verbose='auto',
          validation_data=(fake_data_val, fake_labels_val),
          callbacks=[logging_callback]
         )

Epoch 1/100
1/8 [==>...........................] - ETA: 16s - loss: 0.7040 - accuracy: 0.5000

wandb: Adding directory to artifact (/content/wandb/run-20230826_195117-fazow6bv/files/model-best)... Done. 0.0s


8/8 [==============================] - 4s 252ms/step - loss: 0.6973 - accuracy: 0.4860 - val_loss: 0.6948 - val_accuracy: 0.5010
Epoch 2/100
8/8 [==============================] - 0s 13ms/step - loss: 0.6905 - accuracy: 0.5340 - val_loss: 0.6960 - val_accuracy: 0.5060
Epoch 3/100
8/8 [==============================] - 0s 17ms/step - loss: 0.6817 - accuracy: 0.5870 - val_loss: 0.6977 - val_accuracy: 0.4840
Epoch 4/100
8/8 [==============================] - 0s 20ms/step - loss: 0.6744 - accuracy: 0.5900 - val_loss: 0.7046 - val_accuracy: 0.5130
Epoch 5/100
8/8 [==============================] - 0s 43ms/step - loss: 0.6684 - accuracy: 0.5940 - val_loss: 0.7047 - val_accuracy: 0.5050
Epoch 6/100
8/8 [==============================] - 0s 12ms/step - loss: 0.6608 - accuracy: 0.6220 - val_loss: 0.7068 - val_accuracy: 0.4900
Epoch 7/100
8/8 [==============================] - 0s 13ms/step - loss: 0.6535 - accuracy: 0.6440 - val_loss: 0.7123 - val_accuracy: 0.4880
Epoch 8/100
8/8 [==============

## PyTorch

In [48]:
pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.0/725.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 kB 45.6 MB/s eta 0:00:00


In [49]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger

In [50]:
wandb_logger = WandbLogger(project="example-DL-Book", name="run-2", entity="tpalczew")

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:397: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


In [51]:
wandb.init(project="example-DL-Book", name="run-2")
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 100
}

wandb_logger = WandbLogger(project="example-DL-Book")
trainer = pl.Trainer(logger=wandb_logger)

class LitModule(pl.LightningModule):
    def __init__(self, *args, **kwarg):
        self.save_hyperparameters()

    def training_step(self, batch, batch_idx):
        self.log("train/loss", loss)

accuracy,▁▂▃▃▄▄▄▅▆▆▆▇▇▇▇█████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▇▇▆▆▅▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,█▄▅▇▆▄▁██▅▇▄▄▅▅▅▃▅▃▃▃▄▃▄▃▃▂▂▃▄▂▂▃▄▂▃▃▃▂▃
val_loss,▁▁▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
accuracy,1.0
best_epoch,0
best_val_loss,0.69482
epoch,99
loss,0.0051
val_accuracy,0.48


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


https://wandb.ai/site/articles/pytorch-lightning-with-weights-biases